In [1]:
%cd '/content/drive/MyDrive/Advanced Research Methods/Homework 2'

/content/drive/MyDrive/Advanced Research Methods/Homework 2


In [2]:
import pandas as pd
import random, math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import fftpack
import torch.utils.data as utils
import os
from pathlib import Path
from sklearn.metrics import mean_absolute_percentage_error
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [3]:
def reshape(root):

 tps_df = pd.read_pickle(root)

 reshaped_tps_df = pd.DataFrame()
 reshaped_tps_df['TIME'] = tps_df.time.unique()
 for seg in tps_df.segmentID.unique():
     column = tps_df[tps_df['segmentID'] == seg][['time','TrafficIndex_GP']].drop_duplicates(subset=['time'])
     column.columns = ['TIME', str(seg)]
     reshaped_tps_df = reshaped_tps_df.join(column.set_index('TIME'), on='TIME')

 reshaped_tps_df = reshaped_tps_df.set_index('TIME')
 return reshaped_tps_df



In [4]:
train_data = reshape('tps_df.pkl')


In [5]:
class TrafficForecast(Dataset):
    def __init__(self,df, window, horizon):
        self.window = window
        self.horizon = horizon
        self.df = df
        self.inputs = []
        self.targets = []

        # df = pd.read_pickle(self.pkl_path)
        # df['time'] = pd.to_datetime(df['time'])
        # df['unix_timestamp'] = df['time'].astype(int) / 10**9
        # df['dow'] = df['time'].dt.weekday
        # df['hour'] = df['time'].dt.hour
        # # df['min'] = df['time'].dt.minute
        # self.df = df.sample(frac=1).reset_index(drop=True)
        # self.seg_ids = self.df['segmentID'].unique()

        self.setup_forecast()

    def setup_forecast(self):
        for segid in self.df.columns:
          self.df = self.df.fillna(method="ffill")
          TI_series = self.df[segid].values

          for t in range(0,len(TI_series)-(self.window+self.horizon)):
              x = TI_series[t:t+self.window]
              y = TI_series[t+self.window:(t+self.window+self.horizon)]


              self.inputs.append(x)
              self.targets.append(y)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(self.window,1)
        y = torch.tensor(self.targets[idx],dtype=torch.float32)

        return {'inputs':X,'outputs':y}


In [6]:
tobj = TrafficForecast(train_data,36,12)


In [7]:
for idx, data in enumerate(tobj):
  print (data['inputs'].shape, data['outputs'].shape)
  if idx == 0:
    break


torch.Size([36, 1]) torch.Size([12])


In [8]:
bs = 200
dataloader_train = DataLoader(tobj, batch_size=bs, shuffle=False, num_workers=4,drop_last=True)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [9]:
from models import MyLSTM, MyGRU, MyRNNCell, MyRNN

In [10]:
model = MyRNN(1, 2, 1,12)
# model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
mean_loss = []
losses = []

n_epochs = 20

for it in tqdm(range(n_epochs)):
  # zero the parameter gradients
  for i_batch, sample_batched in enumerate(dataloader_train):
    optimizer.zero_grad()
    outputs = model(sample_batched['inputs'])
    loss = criterion(outputs, sample_batched['outputs'])
    losses.append(loss.item())
    loss.backward()
    optimizer.step()



  mean_loss.append(np.mean(losses))

  if (it+1) % 1 == 0:
    print(f'Epoch {it+1}/{n_epochs}, Training Loss: {np.mean(losses):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
  5%|▌         | 1/20 [01:53<35:55, 113.43s/it]

Epoch 1/20, Training Loss: 0.0066


 10%|█         | 2/20 [03:48<34:20, 114.46s/it]

Epoch 2/20, Training Loss: 0.0065


 15%|█▌        | 3/20 [05:41<32:14, 113.77s/it]

...

Epoch 18/20, Training Loss: 0.0062


 95%|█████████▌| 19/20 [36:10<01:53, 113.46s/it]

Epoch 19/20, Training Loss: 0.0062


100%|██████████| 20/20 [38:07<00:00, 114.36s/it]

Epoch 20/20, Training Loss: 0.0062


In [40]:
torch.save(model.state_dict(), 'rnnmodel.pt')


In [41]:
class Test(Dataset):
    def __init__(self,df):

        self.df = df
        self.inputs = []

        self.setup_forecast()

    def setup_forecast(self):


        for segid in self.df.columns:
          self.df = self.df.fillna(method="ffill")
          TI_series = self.df[segid].values
          self.inputs.append(TI_series)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        X = torch.tensor(self.inputs[idx],dtype=torch.float32).reshape(-1,1)


        return {'inputs':X}

In [42]:
def run_files(i,tobj):

    pkl_file= (pkl_files[i])
    t_value = reshape(os.path.join('Test',pkl_file))
    tobj_ = Test(t_value)
    dfp = pd.DataFrame()
    for idx,data in enumerate(tobj_):
      input_data = data['inputs'].reshape(-1,36,1)
      preds = model(input_data).detach()
      # preds = model(input_data).detach().cpu().numpy()[0]

      dfp[idx] = preds
    return dfp




In [43]:
pkl_files = os.listdir('Test')
print(pkl_files)

['tps_6.pkl', 'tps_9.pkl', 'tps_2.pkl', 'tps_14.pkl', 'tps_4.pkl', 'tps_5.pkl', 'tps_3.pkl', 'tps_12.pkl', 'tps_7.pkl', 'tps_13.pkl', 'tps_10.pkl', 'tps_1.pkl', 'tps_11.pkl', 'tps_15.pkl', 'tps_8.pkl']


In [44]:
op0 = run_files(0,tobj)
op0.index = pd.date_range(start='2020-06-07 11:15:00', end='2020-06-07 14:00:00', freq='15min').astype(int) / 10**9
op0.columns = train_data.columns

op1 = run_files(1,tobj)
op1.index = pd.date_range(start='2020-06-10 14:15:00', end='2020-06-10 17:00:00', freq='15min').astype(int) / 10**9
op1.columns = train_data.columns

op2 = run_files(2,tobj)
op2.index = pd.date_range(start='2020-06-03 7:15:00', end='2020-06-03 10:00:00', freq='15min').astype(int) / 10**9
op2.columns = train_data.columns

op3 = run_files(3,tobj)
op3.index = pd.date_range(start='2020-06-15 19:15:00', end='2020-06-15 22:00:00', freq='15min').astype(int) / 10**9
op3.columns = train_data.columns


op4 = run_files(4,tobj)
op4.index = pd.date_range(start='2020-06-05 9:15:00', end='2020-06-05 12:00:00', freq='15min').astype(int) / 10**9
op4.columns = train_data.columns

op5 = run_files(5,tobj)
op5.index = pd.date_range(start='2020-06-06 10:15:00', end='2020-06-06 13:00:00', freq='15min').astype(int) / 10**9
op5.columns = train_data.columns

op6 = run_files(6,tobj)
op6.index = pd.date_range(start='2020-06-04 8:15:00', end='2020-06-04 11:00:00', freq='15min').astype(int) / 10**9
op6.columns = train_data.columns

op7 = run_files(7,tobj)
op7.index = pd.date_range(start='2020-06-13 17:15:00', end='2020-06-13 20:00:00', freq='15min').astype(int) / 10**9
op7.columns = train_data.columns

op8 = run_files(8,tobj)
op8.index = pd.date_range(start='2020-06-08 12:15:00', end='2020-06-08 15:00:00', freq='15min').astype(int) / 10**9
op8.columns = train_data.columns

op9 = run_files(9,tobj)
op9.index = pd.date_range(start='2020-06-14 18:15:00', end='2020-06-14 21:00:00', freq='15min').astype(int) / 10**9
op9.columns = train_data.columns

op10 = run_files(10,tobj)
op10.index = pd.date_range(start='2020-06-11 15:15:00', end='2020-06-11 18:00:00', freq='15min').astype(int) / 10**9
op10.columns = train_data.columns

op11 = run_files(11,tobj)
op11.index = pd.date_range(start='2020-06-02 6:15:00', end='2020-06-02 9:00:00', freq='15min').astype(int) / 10**9
op11.columns = train_data.columns

op12 = run_files(12,tobj)
op12.index = pd.date_range(start='2020-06-12 16:15:00', end='2020-06-12 19:00:00', freq='15min').astype(int) / 10**9
op12.columns = train_data.columns

op13 = run_files(13,tobj)
op13.index = pd.date_range(start='2020-06-16 20:15:00', end='2020-06-16 23:00:00', freq='15min').astype(int) / 10**9
op13.columns = train_data.columns

op14 = run_files(14,tobj)
op14.index = pd.date_range(start='2020-06-09 13:15:00', end='2020-06-09 16:00:00', freq='15min').astype(int) / 10**9
op14.columns = train_data.columns


final_output = pd.concat([op11,op2,op6,op4,op5,op0,op8,op14,op1,op10,op12,op7,op9,op3,op13])

In [45]:
print(final_output.shape)

(180, 87)


In [46]:
print(final_output)

                    11        12        13        14        15        16  \
1.591078e+09  0.978745  0.976757  0.902184  0.919796  0.600262  0.837607   
1.591079e+09  0.977844  0.975778  0.905884  0.921057  0.645772  0.850249   
1.591080e+09  0.977555  0.975469  0.910468  0.923495  0.687150  0.862703   
1.591081e+09  0.977689  0.975635  0.915665  0.926832  0.724223  0.874718   
1.591082e+09  0.978026  0.976030  0.921145  0.930601  0.759031  0.886471   
...                ...       ...       ...       ...       ...       ...   
1.592345e+09  0.999566  0.999566  0.994860  0.999566  0.999566  0.996742   
1.592346e+09  0.999674  0.999674  0.995175  0.999674  0.999674  0.996975   
1.592347e+09  0.999703  0.999703  0.995223  0.999703  0.999703  0.997015   
1.592348e+09  0.999824  0.999824  0.995537  0.999824  0.999824  0.997252   
1.592348e+09  0.999840  0.999840  0.995556  0.999840  0.999840  0.997270   

                    17        18        19        20  ...        94        95  \
1.5910

In [47]:
final_output.to_json('final_output_rnn.json')

In [48]:
pd.read_json('final_output_rnn.json')

<ipython-input-48-fb9a8b4ecf12>:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.read_json('final_output_rnn.json')


,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
2020-06-02 06:15:28,0.978745,0.976757,0.902184,0.919796,0.600262,0.837607,0.857735,0.995641,1.000611,0.990672,...,1.000611,1.000611,1.000611,1.000611,0.991666,1.000611,1.000611,1.000611,1.000611,1.000611
2020-06-02 06:30:24,0.977844,0.975778,0.905884,0.921057,0.645772,0.850249,0.867589,0.995403,1.000567,0.990238,...,1.000567,1.000567,1.000567,1.000567,0.991271,1.000567,1.000567,1.000567,1.000567,1.000567
2020-06-02 06:45:20,0.977555,0.975469,0.910468,0.923495,0.687150,0.862703,0.877591,0.995285,1.000500,0.990070,...,1.000500,1.000500,1.000500,1.000500,0.991113,1.000500,1.000500,1.000500,1.000500,1.000500
2020-06-02 07:00:16,0.977689,0.975635,0.915665,0.926832,0.724223,0.874718,0.887480,0.995146,1.000281,0.990012,...,1.000281,1.000281,1.000281,1.000281,0.991039,1.000281,1.000281,1.000281,1.000281,1.000281
2020-06-02 07:15:12,0.978026,0.976030,0.921145,0.930601,0.759031,0.886471,0.897278,0.994990,0.999979,0.990000,...,0.999979,0.999979,0.999979,0.999979,0.990998,0.999979,0.999979,0.999979,0.999979,0.999979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16 22:00:32,0.999566,0.999566,0.994860,0.999566,0.999566,0.996742,0.950619,0.943335,0.981682,0.986388,...,0.999566,0.999566,0.999566,0.999566,0.999566,0.999566,0.999566,0.999566,0.999566,0.999566
2020-06-16 22:15:28,0.999674,0.999674,0.995175,0.999674,0.999674,0.996975,0.952890,0.946399,0.982580,0.987078,...,0.999674,0.999674,0.999674,0.999674,0.999674,0.999674,0.999674,0.999674,0.999674,0.999674
2020-06-16 22:30:24,0.999703,0.999703,0.995223,0.999703,0.999703,0.997015,0.953120,0.947510,0.982682,0.987161,...,0.999703,0.999703,0.999703,0.999703,0.999703,0.999703,0.999703,0.999703,0.999703,0.999703
2020-06-16 22:45:20,0.999824,0.999824,0.995537,0.999824,0.999824,0.997252,0.955234,0.950199,0.983532,0.987819,...,0.999824,0.999824,0.999824,0.999824,0.999824,0.999824,0.999824,0.999824,0.999824,0.999824
